# CRITICAL PATH METHOD (CPM)

<u>**Problem**:</u> You have a project consisting of various activities that need to be completed.
        Each activity has a duration, and there can be dependencies between activities.
        The DataFrame contains three columns:<br><br><br>

**"Activities":** This column represents the names or identifiers of the activities.<br><br>
**"Predecessor":** This column specifies the activities that need to be completed before a particular activity can start.<br>
    It can contain multiple predecessors separated by commas.<br><br>
**"Durations":** This column represents the duration (in some unit of time) required to complete each activity.<br>
    You need to find the critical path of the project, which is the sequence of activities that has the longest total duration.<br>
    The critical path determines the minimum time required to complete the entire project.<br>
    Additionally, you need to calculate the minimum durations to complete all the activities.<br>

Import Libraries 

In [1]:
import pandas as pd
import numpy as np


In [2]:

def find_critical_path(df):
    # Create a dictionary to store the activity durations and another dict to store the predecessors for each activity
    durations = {}
    predecessors = {}

    # Iterate over the DataFrame rows and populate the durations and predecessors dictionaries
    for index, row in df.iterrows():
        activity = row['Activities']
        duration = row['Durations']
        predecessors[activity] = row['Predecessor'].split(',') if pd.notnull(row['Predecessor']) else []
        durations[activity] = duration

    # Create a dictionary to store the earliest start time for each activity
    earliest_start = {}

    # Initialize the earliest start time for the first activity as 0
    earliest_start[df.iloc[0]['Activities']] = 0

    # Iterate over the DataFrame rows to calculate the earliest start time for each activity
    for index, row in df.iterrows():
        activity = row['Activities']

        if activity not in earliest_start:
            earliest_start[activity] = max([earliest_start[p] + durations[p] for p in predecessors[activity]])

    # Create a dictionary to store the latest start time for each activity
    latest_start = {}

    # Initialize the latest start time for the last activity as the earliest start time
    latest_start[df.iloc[-1]['Activities']] = earliest_start[df.iloc[-1]['Activities']]

    # Iterate over the DataFrame rows in reverse order to calculate the latest start time for each activity
    for index, row in df[::-1].iterrows():
        activity = row['Activities']

        if activity not in latest_start:
            successors = [s for s in df[df['Predecessor'].str.contains(activity, na=False)]['Activities']]

            if len(successors) > 0:
                latest_start[activity] = min([latest_start[s] - durations[activity] for s in successors])
            else:
                latest_start[activity] = earliest_start[df.iloc[-1]['Activities']] - durations[activity]

    # Create a dictionary to store the critical path and its duration
    critical_path = {}

    # Iterate over the DataFrame rows to determine the critical path and its duration
    for index, row in df.iterrows():
        activity = row['Activities']
        duration = row['Durations']
        slack = latest_start[activity] - earliest_start[activity]

        if slack == 0:
            critical_path[activity] = duration

    # Calculate the minimum duration to complete all activities
    min_duration = sum(critical_path.values())

    return critical_path, min_duration


Test Case 1

In [3]:
# Example usage
df = pd.DataFrame({
    'Activities': ['A', 'B', 'C', 'D', 'E', 'F', 'G'],
    'Predecessor': [np.nan, 'A', 'A', 'B,C', 'C', 'D', 'E'],
    'Durations': [5, 3, 2, 4, 6, 7, 4]
})

critical_path, min_duration = find_critical_path(df)
print("Critical Path:", critical_path)
print("Minimum Duration:", min_duration)


Critical Path: {'E': 6, 'G': 4}
Minimum Duration: 10


Test Case 2

In [4]:
# Example usage
df = pd.DataFrame({
    'Activities': ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H'],
    'Predecessor': [np.nan, 'A', 'A', 'B', 'C', 'C', 'D,E,F','G'],
    'Durations': [7, 9, 3, 8, 5, 4, 2, 1]
})

critical_path, min_duration = find_critical_path(df)
print("Critical Path:", critical_path)
print("Minimum Duration:", min_duration)


Critical Path: {'A': 7, 'B': 9, 'D': 8, 'G': 2, 'H': 1}
Minimum Duration: 27
